In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)
cf.go_offline();
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

pd.set_option("display.max_columns",None)

In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Ashmina
[nltk_data]     Dangat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
df_spread = pd.read_csv('../dataset/amul_lite_spread.csv')
df_tea = pd.read_csv('../dataset/dairymix_tea.csv')
df_paneer = pd.read_csv('../dataset/paneer.csv')
df_chash = pd.read_csv('../dataset/mother_dairy_tadka_chash.csv')
df_cheese = pd.read_csv('../dataset/motherdairy_cheese.csv')
df_sandwich_bread = pd.read_csv('../dataset/sandwich_bread.csv')
df_whole_wheat_bread = pd.read_csv('../dataset/whole_wheat_bread.csv')
df_ragi_loaf = pd.read_csv('../dataset/ragi_loaf.csv')
df_garlic_bread = pd.read_csv('../dataset/garlic_bread.csv')

In [7]:
df_spread['product_name'] = "Amul lite spread"
df_tea['product_name'] = "Premium Dairy Mix"
df_paneer['product_name'] = "Amul Fresh Paneer"
df_chash['product_name'] = "Mother Dairy Tadka Chas"
df_cheese['product_name'] = "Mother Dairy Cheese"
df_sandwich_bread['product_name'] = "Sandwich bread"
df_whole_wheat_bread['product_name'] = "Whole wheat bread"
df_ragi_loaf['product_name'] = "Ragi Loaf"
df_garlic_bread['product_name'] = "Garlic bread"

In [8]:
df_dairy = pd.concat([df_spread,df_tea,df_paneer,df_chash,df_cheese,df_sandwich_bread,df_whole_wheat_bread,df_ragi_loaf,df_garlic_bread])

In [9]:
df_dairy.head()

Star-rating              Name  \
0  4.0 out of 5 stars  SUMMAIYAH VIRANI   
1  4.0 out of 5 stars              Yash   
2  5.0 out of 5 stars           MUSHTAK   
3  5.0 out of 5 stars     Milind Bondre   
4  5.0 out of 5 stars     Shubham Yadav   

                                Reviewer Profile Url  \
0  https://www.amazon.in/gp/customer-reviews/R1HJ...   
1  https://www.amazon.in/gp/customer-reviews/RTT6...   
2  https://www.amazon.in/gp/customer-reviews/R1PN...   
3  https://www.amazon.in/gp/customer-reviews/R2OT...   
4  https://www.amazon.in/gp/customer-reviews/RFSI...   

                       Review Title  \
0                              Good   
1                              Nice   
2   Reasonable rate with good taste   
3                The healthy butter   
4      Fatima ka dud pita hei india   

                                      Review Content images  \
0                                               Good    NaN   
1  Ofcourse the normal amul butter is much more t...    NaN   
2                        I liked the taste and price    NaN   
3  It is very close to the taste of Amul butter, ...    NaN   
4                                               Best    NaN   

                                        profile icon  \
0  https://images-eu.ssl-images-amazon.com/images...   
1  https://images-na.ssl-images-amazon.com/images...   
2  https://images-na.ssl-images-amazon.com/images...   
3  https://images-na.ssl-images-amazon.com/images...   
4  https://images-na.ssl-images-amazon.com/images...   

                                       Date      product_name helpful  \
0  Reviewed in India 🇮🇳 on 13 February 2023  Amul lite spread     NaN   
1    Reviewed in India 🇮🇳 on 10 August 2022  Amul lite spread     NaN   
2    Reviewed in India 🇮🇳 on 27 August 2022  Amul lite spread     NaN   
3    Reviewed in India 🇮🇳 on 11 August 2021  Amul lite spread     NaN   
4   Reviewed in India 🇮🇳 on 10 October 2021  Amul lite spread     NaN   

  verified variant  
0      NaN     NaN  
1      NaN     NaN  
2      NaN     NaN  
3      NaN     NaN  
4      NaN     NaN

In [10]:
df_dairy = df_dairy.drop(["Review Title"],axis=1)

In [11]:
df_dairy = df_dairy.drop(["Reviewer Profile Url","images","profile icon"],axis=1)

In [12]:
df_dairy.head()

Star-rating              Name  \
0  4.0 out of 5 stars  SUMMAIYAH VIRANI   
1  4.0 out of 5 stars              Yash   
2  5.0 out of 5 stars           MUSHTAK   
3  5.0 out of 5 stars     Milind Bondre   
4  5.0 out of 5 stars     Shubham Yadav   

                                      Review Content  \
0                                               Good   
1  Ofcourse the normal amul butter is much more t...   
2                        I liked the taste and price   
3  It is very close to the taste of Amul butter, ...   
4                                               Best   

                                       Date      product_name helpful  \
0  Reviewed in India 🇮🇳 on 13 February 2023  Amul lite spread     NaN   
1    Reviewed in India 🇮🇳 on 10 August 2022  Amul lite spread     NaN   
2    Reviewed in India 🇮🇳 on 27 August 2022  Amul lite spread     NaN   
3    Reviewed in India 🇮🇳 on 11 August 2021  Amul lite spread     NaN   
4   Reviewed in India 🇮🇳 on 10 October 2021  Amul lite spread     NaN   

  verified variant  
0      NaN     NaN  
1      NaN     NaN  
2      NaN     NaN  
3      NaN     NaN  
4      NaN     NaN

In [13]:
df_dairy = df_dairy.drop(["helpful","verified","variant"],axis=1)

In [14]:
def convert_score(string):
  li = list(string.split(" "))
  return li[0]

In [15]:
df_dairy['Star-rating'] = df_dairy['Star-rating'].apply(convert_score)

In [17]:
df_dairy.head()

Star-rating              Name  \
0         4.0  SUMMAIYAH VIRANI   
1         4.0              Yash   
2         5.0           MUSHTAK   
3         5.0     Milind Bondre   
4         5.0     Shubham Yadav   

                                      Review Content  \
0                                               Good   
1  Ofcourse the normal amul butter is much more t...   
2                        I liked the taste and price   
3  It is very close to the taste of Amul butter, ...   
4                                               Best   

                                       Date      product_name  
0  Reviewed in India 🇮🇳 on 13 February 2023  Amul lite spread  
1    Reviewed in India 🇮🇳 on 10 August 2022  Amul lite spread  
2    Reviewed in India 🇮🇳 on 27 August 2022  Amul lite spread  
3    Reviewed in India 🇮🇳 on 11 August 2021  Amul lite spread  
4   Reviewed in India 🇮🇳 on 10 October 2021  Amul lite spread

In [18]:
!pip install -U textblob

You should consider upgrading via the 'C:\Python 310\python.exe -m pip install --upgrade pip' command.


In [19]:
from textblob import TextBlob

In [20]:
df_dairy['Review Content']= df_dairy['Review Content'].astype(str)
pol = lambda x: TextBlob(x).sentiment.polarity
df_dairy['polarity'] = df_dairy['Review Content'].apply(pol)

In [21]:
df_dairy.head()

Star-rating              Name  \
0         4.0  SUMMAIYAH VIRANI   
1         4.0              Yash   
2         5.0           MUSHTAK   
3         5.0     Milind Bondre   
4         5.0     Shubham Yadav   

                                      Review Content  \
0                                               Good   
1  Ofcourse the normal amul butter is much more t...   
2                        I liked the taste and price   
3  It is very close to the taste of Amul butter, ...   
4                                               Best   

                                       Date      product_name  polarity  
0  Reviewed in India 🇮🇳 on 13 February 2023  Amul lite spread  0.700000  
1    Reviewed in India 🇮🇳 on 10 August 2022  Amul lite spread  0.166667  
2    Reviewed in India 🇮🇳 on 27 August 2022  Amul lite spread  0.600000  
3    Reviewed in India 🇮🇳 on 11 August 2021  Amul lite spread  0.270833  
4   Reviewed in India 🇮🇳 on 10 October 2021  Amul lite spread  1.000000

In [22]:
sub = lambda x: TextBlob(x).sentiment.subjectivity
df_dairy['subjectivity'] = df_dairy['Review Content'].apply(sub)
df_dairy.sample(10)

Star-rating              Name  \
38         5.0            Aparna   
0          4.0         Vinamrata   
47         5.0       Ankit Anant   
54         4.0         Pkaourora   
29         4.0  Siddharth Hiwale   
13         4.0           KANCHAN   
96         1.0        sonu kumar   
18         4.0          Satishji   
23         2.0              Amit   
26         4.0   Amazon Customer   

                                       Review Content  \
38  Its just best the packaging flavour evrythng i...   
0                                                Good   
47                                        I loved it.   
54                One of the best among other options   
29                In one word - Quantity with Quality   
13                                            Like it   
96                                  Old date received   
18  Very good taste and mouth feel. But please not...   
23  Paneer was hard unlike fresh paneer purchased ...   
26                                               good   

                                        Date             product_name  \
38    Reviewed in India 🇮🇳 on 17 August 2020      Mother Dairy Cheese   
0   Reviewed in India 🇮🇳 on 28 February 2023        Whole wheat bread   
47      Reviewed in India 🇮🇳 on 15 June 2020      Mother Dairy Cheese   
54       Reviewed in India 🇮🇳 on 5 June 2021        Amul Fresh Paneer   
29      Reviewed in India 🇮🇳 on 7 April 2022  Mother Dairy Tadka Chas   
13       Reviewed in India 🇮🇳 on 29 May 2021      Mother Dairy Cheese   
96      Reviewed in India 🇮🇳 on 27 July 2021  Mother Dairy Tadka Chas   
18      Reviewed in India 🇮🇳 on 17 June 2022  Mother Dairy Tadka Chas   
23   Reviewed in India 🇮🇳 on 19 January 2023        Amul Fresh Paneer   
26   Reviewed in India 🇮🇳 on 21 October 2020      Mother Dairy Cheese   

    polarity  subjectivity  
38  0.955000      0.540000  
0   0.700000      0.600000  
47  0.700000      0.800000  
54  0.437500      0.337500  
29  0.000000      0.000000  
13  0.000000      0.000000  
96  0.100000      0.200000  
18  0.455000      0.890000  
23  0.004167      0.520833  
26  0.700000      0.600000

In [23]:
def lowercase(string):
  string = re.sub("[^a-zA-Z]",' ',string)
  string = string.lower()
  return string

In [24]:
df_dairy['Review Content'] = df_dairy['Review Content'].apply(lowercase)

In [25]:
df_dairy.head()

Star-rating              Name  \
0         4.0  SUMMAIYAH VIRANI   
1         4.0              Yash   
2         5.0           MUSHTAK   
3         5.0     Milind Bondre   
4         5.0     Shubham Yadav   

                                      Review Content  \
0                                               good   
1  ofcourse the normal amul butter is much more t...   
2                        i liked the taste and price   
3  it is very close to the taste of amul butter  ...   
4                                               best   

                                       Date      product_name  polarity  \
0  Reviewed in India 🇮🇳 on 13 February 2023  Amul lite spread  0.700000   
1    Reviewed in India 🇮🇳 on 10 August 2022  Amul lite spread  0.166667   
2    Reviewed in India 🇮🇳 on 27 August 2022  Amul lite spread  0.600000   
3    Reviewed in India 🇮🇳 on 11 August 2021  Amul lite spread  0.270833   
4   Reviewed in India 🇮🇳 on 10 October 2021  Amul lite spread  1.000000   

   subjectivity  
0      0.600000  
1      0.436667  
2      0.800000  
3      0.296667  
4      0.300000

In [26]:
!pip install vaderSentiment

You should consider upgrading via the 'C:\Python 310\python.exe -m pip install --upgrade pip' command.


In [28]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
df_dairy[['polarity','subjectivity']] = df_dairy['Review Content'].apply(lambda Text:pd.Series(TextBlob(Text).sentiment))

for index,row in df_dairy['Review Content'].iteritems():
  score = SentimentIntensityAnalyzer().polarity_scores(row)
  neg = score['neg']
  neu = score['neu']
  pos = score['pos']
  if neg>pos:
    df_dairy.loc[index,'sentiment'] = "Negative"
  elif pos>neg:
    df_dairy.loc[index,'sentiment'] = "Positive"
  else:
    df_dairy.loc[index,'sentiment'] = "Neutral"

In [29]:
df_dairy.head()

Star-rating              Name  \
0         4.0  SUMMAIYAH VIRANI   
1         4.0              Yash   
2         5.0           MUSHTAK   
3         5.0     Milind Bondre   
4         5.0     Shubham Yadav   

                                      Review Content  \
0                                               good   
1  ofcourse the normal amul butter is much more t...   
2                        i liked the taste and price   
3  it is very close to the taste of amul butter  ...   
4                                               best   

                                       Date      product_name  polarity  \
0  Reviewed in India 🇮🇳 on 13 February 2023  Amul lite spread  0.700000   
1    Reviewed in India 🇮🇳 on 10 August 2022  Amul lite spread  0.166667   
2    Reviewed in India 🇮🇳 on 27 August 2022  Amul lite spread  0.600000   
3    Reviewed in India 🇮🇳 on 11 August 2021  Amul lite spread  0.270833   
4   Reviewed in India 🇮🇳 on 10 October 2021  Amul lite spread  1.000000   

   subjectivity sentiment  
0      0.600000   Neutral  
1      0.436667  Positive  
2      0.800000  Positive  
3      0.296667  Positive  
4      0.300000  Positive

In [30]:
import pickle

In [31]:
file = open('sentiment.pkl','wb')
pickle.dump(df_dairy,file)
file.close()

In [35]:
df_dairy.loc[df_dairy['product_name'] == 'Ragi Loaf']

Star-rating                 Name  \
0          4.0                Pooja   
1          5.0     Michelle D’souza   
2          1.0                   PS   
3          5.0                  CDC   
4          3.0      Amazon Customer   
5          3.0             Suhasini   
6          5.0                   SS   
7          2.0                priti   
8          4.0         Harmeet Kaur   
9          3.0                  AxK   
10         1.0       Prakash Subedi   
11         5.0   Nivedita Narjinari   
12         5.0                A. D.   
13         2.0         Cassi Dsouza   
14         4.0               Batman   
15         1.0                Ashok   
16         1.0      Amazon Customer   
17         1.0                Jay S   
18         5.0       Savita Thukral   
19         1.0       sheeru singhal   
20         1.0      Amazon Customer   
21         1.0        P Bisht Noida   
22         5.0  A Millenials Review   
23         5.0              Emerald   

                                       Review Content  \
0               nice  but one slice had fungus on it    
1   tried this bread for the first time and it was...   
2   there was some blue coloured plastic like thin...   
3   this is a nice bread  its tough not soft as it...   
4   product ok  packaging is bad  hard to store af...   
5   i wish sellers would describe their products a...   
6   genuine ragi loaf with a l il bit of atta  who...   
7                      too hard    din like the taste   
8   a little hard   but still nice  fills you till...   
9   it s wheat bread with small amount of ragi  go...   
10  for a    ragi  it is called ragi bread    you ...   
11  i loved this  i have removed gluten from my di...   
12  non glutonous and filling  has nutritional ben...   
13  it has maida or wheat flour mixed in it  so th...   
14                                               good   
15  this product taste is not good   too chewy to ...   
16  this product is a sham  it s basically a whole...   
17  the bread slices were not fresh at all and so ...   
18                                        great taste   
19                                     it s very dry    
20                                          dry bread   
21  it is clearly written  it has wheat flour and ...   
22  love the bread  love it s size  flavour and he...   
23  loved it  really good alternative to store bro...   

                                      Date product_name  polarity  \
0       Reviewed in India on 28 March 2023    Ragi Loaf  0.600000   
1        Reviewed in India on 6 March 2023    Ragi Loaf  0.562500   
2        Reviewed in India on 8 April 2023    Ragi Loaf  0.000000   
3      Reviewed in India on 22 August 2022    Ragi Loaf -0.115432   
4         Reviewed in India on 16 May 2022    Ragi Loaf -0.163889   
5      Reviewed in India on 9 October 2020    Ragi Loaf  0.132479   
6         Reviewed in India on 4 June 2021    Ragi Loaf  0.215556   
7      Reviewed in India on 19 August 2022    Ragi Loaf -0.291667   
8    Reviewed in India on 28 February 2021    Ragi Loaf  0.040278   
9       Reviewed in India on 29 April 2021    Ragi Loaf -0.016667   
10        Reviewed in India on 29 May 2022    Ragi Loaf  0.093750   
11      Reviewed in India on 16 April 2021    Ragi Loaf  0.750000   
12   Reviewed in India on 17 February 2021    Ragi Loaf  0.000000   
13         Reviewed in India on 5 May 2021    Ragi Loaf  0.000000   
14    Reviewed in India on 20 January 2020    Ragi Loaf  0.700000   
15         Reviewed in India on 7 May 2021    Ragi Loaf -0.068750   
16      Reviewed in India on 28 April 2021    Ragi Loaf  0.066667   
17   Reviewed in India on 18 December 2019    Ragi Loaf -0.058333   
18       Reviewed in India on 6 March 2020    Ragi Loaf  0.800000   
19      Reviewed in India on 27 April 2021    Ragi Loaf -0.086667   
20      Reviewed in India on 22 March 2021    Ragi Loaf -0.066667   
21      Reviewed in India on 13 April 2022    Ragi Loaf  0.100000   
22      Review